In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
df = pd.read_csv('slash.txt',sep='\t')


In [59]:
G = nx.DiGraph()
for index,rows in df.iterrows():
    u = rows['FromNodeId']
    v = rows['ToNodeId']
    s = rows['Sign']
    G.add_edge(u,v,sign=s)

In [134]:
out_pos = {}
out_neg = {}
for nodes in G.nodes():
    pos = 0
    neg = 0
    l = dict(G[nodes])
    for keys in l:
        if(l[keys]['sign'] == 1):
            pos += 1
        else:
            neg += 1
    out_pos[nodes] = pos
    out_neg[nodes] = neg

In [63]:
nx.set_node_attributes(G,out_pos,'out_pos')
nx.set_node_attributes(G,out_neg,'out_neg')


In [64]:
Nodes = G.nodes(data=True)

In [65]:
pred_sign = {}
for edge in G.edges():
    u = edge[0]
    if Nodes[u]['out_pos']>= Nodes[u]['out_neg']:
        pred_sign[edge] = 1
    else:
        pred_sign[edge] = -1
    
    
    

In [66]:
nx.set_edge_attributes(G, pred_sign, 'out_index_sign')

In [102]:
out_index = nx.get_edge_attributes(G,'out_index_sign')
sign = nx.get_edge_attributes(G,'sign')
embed = nx.get_edge_attributes(G,'embed')

df = pd.DataFrame(columns=['edge', "out_index_sign", 'sign', 'embed', "in_index_sign"])

In [106]:
embed = {}
for edge in list(G.edges()):
    inward_i=[]
    inward_j=[]
    outward_i=[]
    outward_j=[]
    for items in list(G.in_edges(edge[0])):
        inward_i.append(items[0])
    for items in list(G.in_edges(edge[1])):
        inward_j.append(items[0])
    for keys in dict(G[edge[0]]):
        outward_i.append(keys)
    for keys in dict(G[edge[1]]):
        outward_j.append(keys)
    final_i = list(set(inward_i) | set(inward_j))
    final_j = list(set(outward_i) | set(outward_j))
    final = list(set(final_i) & set(final_j)) 
    embed[edge] = len(final)
    
    

In [107]:
nx.set_edge_attributes(G, embed, 'embed')

In [108]:
df['edge'] = out_index.keys()
df['out_index_sign'] = out_index.values()
df['sign'] = sign.values()
df['embed'] = embed.values()

In [109]:
y_pred = df["out_index_sign"]
y = df["sign"]

In [110]:
from sklearn.metrics import accuracy_score
accuracy_score( y,y_pred)

0.8572055455005626

In [111]:
#in edge heuristics
in_pos = {}
in_neg = {}

for nodes in G.nodes():
    l = list(G.in_edges(nodes,data=True))
    pos=[]
    neg=[]
    for items in l:
        if items[2]['sign'] == 1:
            pos.append(items[0])
        else:
            neg.append(items[0])
    in_pos[nodes] = pos
    in_neg[nodes] = neg

nx.set_node_attributes(G,in_pos,'in_pos')
nx.set_node_attributes(G,in_neg,'in_neg')



In [129]:
pred_sign_in = {}
for edge in G.edges():
    u = edge[0]
    if Nodes[u]['in_pos']>= Nodes[u]['in_neg']:
        pred_sign_in[edge] = 1
    else:
        pred_sign_in[edge] = -1

        
        
nx.set_edge_attributes(G, pred_sign_in, 'in_index')


In [130]:
in_index = nx.get_edge_attributes(G,'in_index')
sign = nx.get_edge_attributes(G,'sign')
embed = nx.get_edge_attributes(G,'embed')

In [131]:

df['edge'] = in_index.keys()
df['in_index_sign'] = in_index.values()
df['sign'] = sign.values()
df['embed'] = embed.values()

In [132]:
y_pred = df["in_index_sign"]
y = df["sign"]

In [133]:
from sklearn.metrics import accuracy_score
accuracy_score( y,y_pred)

0.5200418061114126